### week 7


In [ ]:
! jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
! source activate example-environment

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as stats 

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import STAMEN_TERRAIN
output_notebook()

import datashader as ds
import datashader.transfer_functions as tf
from datashader.colors import Hot, viridis
from datashader.utils import export_image
from datashader.bokeh_ext import InteractiveImage

from functools import partial

import param
import paramnb
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%%time
df_main = pd.read_csv('yellow_tripdata_2016-06_ed.xz', parse_dates=['tpep_pickup_datetime'])
df_main.set_index('tpep_pickup_datetime', inplace=True)

In [ ]:
df_loc = pd.read_csv('time_region_102_pickup_count.xz')

df_loc = df_loc.set_index('datetime')
df_loc = df_loc['2016-05-31 23:00:00':]
regions_list = df_loc.columns.astype(int)

In [ ]:
# загружаем предсказания количества поездок в июне 2016
predictions = pd.read_csv('predictions_2016-06.xz', index_col=0)

loc = predictions['id'].str.split('_')

regions = []; dates = []; hours = []; forwards = []
for x in loc:
    regions.append(x[0])
    dates.append(x[1])
    hours.append(x[2])
    forwards.append(int(x[3]))

df = pd.DataFrame()
df['regions'] = regions
df['datetime']=  [date+' '+hour for date, hour in zip(dates, hours)]
df['predicts'] = forwards
df['datetime'] = [pd.datetime.strptime(row, '%Y-%m-%d %H') for row in df['datetime']]
df['y'] = predictions['y']
df.set_index('datetime', inplace=True)
df_june_predictions = df

In [ ]:
class GetData(param.Parameterized):
    hour = param.Integer(default=3, bounds=(0,23))
    date = param.ObjectSelector(default='2016-06-01', 
       objects=np.unique([x.strftime('%Y-%m-%d') for x in  df_loc.index.astype('<M8[D]')]),
                check_on_set=True)

In [ ]:
def prep_pred():
    df_pred = df_june_predictions[df_june_predictions.predicts == 1][['regions', 'y']]
    df_pred.columns = ['regions', 'pickups']
    return df_pred

In [ ]:
def prep_real():
    t = list()
    ind_list  = list()
    i = 0 
    index = df_loc.index.values
    columns = df_loc.columns
    j = -1
    for row in df_loc.values:
        for i in range(len(columns)):
            t.append([columns[i], row[i]])
            if j < 0:
                k = 0
            else:
                k = j
            ind_list.append(index[k])
        j = j + 1
    df_real = pd.DataFrame(t, index=ind_list, columns = ['regions', 'pickups'])
    return df_real

In [ ]:
regions = pd.read_csv('regions.csv', sep=';')

In [ ]:
def region_to_coordinates(row):
    region = int(row[0])
    pickups = row[1]
    coordinates = regions[regions.region == region].values[0]
    return np.mean([coordinates[3], coordinates[4]]), np.mean([coordinates[1], coordinates[2]]), pickups

In [ ]:
%%time
df_pred = prep_pred().apply(region_to_coordinates,  axis = 1)
df_real = prep_real().apply(region_to_coordinates,  axis = 1)

In [ ]:
def prepare_data_for_map():
    date = str(GetData.date)
    hour = str(GetData.hour)
    if len(hour) == 1:
        hour = '0%s' % hour
    date_str = '%s %s:00:00' %(date, hour)
    
    pred_arr = df_pred[df_pred.index == date_str].values
    real_arr = df_real[df_real.index == date_str].values
    
    return pred_arr, real_arr

In [ ]:
import folium
from folium.plugins import HeatMap
from folium import plugins

In [ ]:
def visualize_map(arr):
    new_york_coordinates = (40.72, -73.90)
    map_ny = folium.Map(width=700,height=500, location=new_york_coordinates, zoom_start=11, detect_retina=True)

    szt = plugins.ScrollZoomToggler() # Turn off mouse scrolling
    map_ny.add_child(szt)
    map_ny._repr_html_()
    map_ny

    HeatMap(arr, radius=12).add_to(map_ny)        
    return map_ny

#### 1) карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени 

In [ ]:
# выберите дату, час и нажмите кнопку
paramnb.Widgets(GetData, button=True, next_n=3)

In [ ]:
arr_real, arr_pred = prepare_data_for_map()
real_map = visualize_map(arr_real)
pred_map = visualize_map(arr_pred)

In [ ]:
print ('Предсказанный спрос')
real_map

In [ ]:
print ('Реальный спрос')
pred_map

#### 2) временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

In [ ]:
class GetCell(param.Parameterized):
    cell = param.ObjectSelector(default=regions_list[0], objects=regions_list, check_on_set=True)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
def prepare_data():
    region = GetCell.cell
    # получаем предсказанное количество поездок в выбранный день и час (из ранее полученной модели) 
    df = df_june_predictions.copy()
    df = df[df['regions'] == str(region)]
    df1 = df[df['predicts'] == int(1)].copy()
    df2 = df[df['predicts'] == int(6)].copy()
    df = pd.concat([df1, df2[-5:]], axis=0)
    prediction_nums = df
    hours = pd.date_range('2016-06-01 00:00:00', periods=720, freq='H')
    prediction_nums.index = hours
    pred_arr = prediction_nums['y'].values
    real_arr = df_loc.loc[:,str(region)].values
    
    return pred_arr, real_arr

In [ ]:
def plot_real_pred(real, pred):
    plt.figure(figsize=(18,6))
    plt.plot(real, 'r', label='predicted')
    plt.plot(pred, 'b', label='real values')
    plt.title('2016 June, cell: {}, MAE: {}, XGBRegressor'.format(region, mean_absolute_error(real_arr, pred_arr)))
    plt.ylabel('pickups')
    plt.xlabel('hours in june 2016')
    plt.legend()
    plt.show()

In [ ]:
# выберите интересующую ячейку (регион) и намите на кнопку
paramnb.Widgets(GetCell, button=True, next_n=1)

In [ ]:
pred_arr, real_arr = prepare_data()
print ('Сравнение первых 100 часов')
plot_real_pred(pred_arr[:100], real_arr[:100])
print ('Сравнение всего июня')
plot_real_pred(pred_arr, real_arr)